# 6b. Generate curb ramp crossings

In [ ]:
# Standard library and path imports
import set_path

# Third-party library imports
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import geopandas as gpd
from shapely import wkt
import momepy
import networkx as nx
from branca.element import Template, MacroElement
import branca.colormap as cm
from tqdm.notebook import tqdm
tqdm.pandas()
import folium

# Local or project-specific imports
import crossing_utils
import plot_utils
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf

## Import walking and road network

In [ ]:
# Get basic pedestrian network with widths
gdf_network = gpd.read_file(cf.output_file_widths).to_crs(crs=st.CRS)
graph_network = momepy.gdf_to_nx(gdf_network, approach='primal')
cc_list = [x for y in zip(gdf_network['sidewalk_id'], gdf_network['sidewalk_id']) for x in y]
gdf_network_nodes = gpd.GeoDataFrame(geometry = gdf_network.boundary.explode(index_parts=True), crs=st.CRS)
gdf_network_nodes['x'], gdf_network_nodes['y'] = gdf_network_nodes.geometry.x, gdf_network_nodes.geometry.y
gdf_network_nodes['cc'] = cc_list
gdf_road_network = gpd.read_file(cf.output_road_network).to_crs(crs=st.CRS)

## Import curb segments and corresponding heights

In [ ]:
# Import curb heights
gdf_ch = gpd.read_file(cf.output_curb_heigts)
graph_ch = momepy.gdf_to_nx(gdf_ch, approach='primal')

## Set maximum curb height to find crossings
- You can run this code for multiple max_height values. These values can be used as user preference input when planning routes. 
- The code generates and saves the most appropriate crossings for each max_height value that was set.
- In practice, settings a maximum curb height up to approximately 0.04m equals a smooth transition from the sidewalk to the street. This is due to noise in the point cloud. \
This parameter increments linearly, so setting max_height at 0.06m would roughly equal a max curb height of 0.02m etc.


In [ ]:
# SET THIS VALUE
max_height = 0.04 # in meter

gdf_ch.dropna(subset=['curb_height'])
gdf_ch = gdf_ch.loc[(gdf_ch['curb_height'] <= max_height)]
gdf_ch['centroid'] = gdf_ch['geometry'].centroid
gdf_ch.reset_index(drop=True, inplace=True)

## Determine connected components of curb segments
These are used as a restrictive rule when creating possible crossing edges (i.e., crosing edges should connect different curbs)

In [ ]:
# Calculate connected components for each edge
# Option 1: From curb height edges
# Option 2: From sidewalk edges
# Option 1 results in more connected components since the curb height edges are more fragmentated (i.e., not all parts of sidewalk egde have height information)

# Calculate using the connected components retrieved from the curb height edges
cc_sidewalks = list(nx.connected_components(graph_ch))
nodes_in_edge = [line.coords[0] for line in gdf_ch['geometry'].to_list()]
cc_ids = [i for node in nodes_in_edge for i in range(len(cc_sidewalks)) if node in cc_sidewalks[i]]
gdf_ch['cc_from_curb_edges'] = cc_ids

# Calculate using the connected components retrieved from sidewalk edges from bgt (from road_curb_segments.ipynb notebook)
gdf_ch['cc_from_sidewalk_edges'] = gdf_ch.groupby('overarching_line_segm').ngroup()

## Obtain valid curb to curb connections based on curb height

In [ ]:
# Get coordinates of crossing feature nodes
gdf_ch['geometry'] = gdf_ch['centroid']
gdf_ch['x'], gdf_ch['y'] = gdf_ch.geometry.x, gdf_ch.geometry.y

# Connect curb height nodes to curb height nodes
min_dist = 0
max_dist = 11
max_connections = 2
gdf_curb_edges = crossing_utils.get_crossing_edges_from_curb_heights(gdf_ch, gdf_ch,
                    min_dist=min_dist, max_dist=max_dist, max_connections=max_connections, crs=st.CRS, cc_column='cc_from_sidewalk_edges')
print('Number of curb to curb connections:', len(gdf_curb_edges))

## Obtain potential crossings

In [ ]:
# Current heuristic: Connect outer points of crossing edge to closest network nodes within max_dis.
# Also, network nodes cannot be part of the same sidewalk.

# Get outer nodes of crossing edges and connect to network
gdf_curb_edge_nodes = gpd.GeoDataFrame(geometry=gdf_curb_edges['geometry'].boundary.explode(index_parts=True), crs=st.CRS).reset_index()
gdf_curb_edge_nodes['x'], gdf_curb_edge_nodes['y'] = gdf_curb_edge_nodes.geometry.x, gdf_curb_edge_nodes.geometry.y
groups = gdf_curb_edge_nodes.groupby(np.arange(len(gdf_curb_edge_nodes.index))//2)

# Loop over possible crossing edges seperately and connect to walking network
gdf_crossings = gpd.GeoDataFrame()
network_to_network = True # Set False if you wish to connect network to curb instead of network to network 

for (idx, frame) in tqdm(groups):
    sub_gdf = crossing_utils.connect_curb_crossing_edge(frame, gdf_network_nodes, graph_network, max_dist=4, 
                                                        max_connections=1, crs=st.CRS, network_to_network=network_to_network)
    gdf_crossings = pd.concat([gdf_crossings, sub_gdf])
gdf_crossings = gdf_crossings.groupby(['geometry']).nth(0)
print('Number of potential crossings:', len(gdf_crossings))

## Filter out crossings that do not cross the road or that cross multiple roads

In [ ]:
# Remove potential crossings that do not cross the road or cross the road multiple times 
# Here, 'road' corresponds to the centerlines extracted from the unary union of car roads, public transport roads and bicycle lanes
gdf_crossings['intersections'] = gdf_crossings.progress_apply(
    lambda row: crossing_utils.count_line_gdf_intersections(row.geometry, gdf_road_network), axis=1)

gdf_crossings = gdf_crossings.loc[gdf_crossings['intersections'] == 1]
gdf_crossings = gdf_crossings.drop(columns=['intersections'])
print('Number of valid crossings:', len(gdf_crossings))

## Store

In [ ]:
gdf_crossings['curb_height_max'] = max_height
gdf_crossings.to_file(cf.output_curb_crossings_base + '_max_height_{}.gpkg'.format(max_height), driver='GPKG')

## Visualize curb ramp crossings

In [ ]:
# set True for satellite background, False for standard background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.7940192423718443], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add colormap
cmp = cm.linear.RdYlGn_11.colors
cmp = list(reversed(cmp))
colormap = cm.LinearColormap(colors=cmp, vmin=0, vmax=0.2, caption='Curb height (m)')
colormap.add_to(map)

# Add network and new edges
geo_j = folium.GeoJson(gdf_network, style_function=lambda x: {"color": "black", "weight": 2}).add_to(map)
geo_j = folium.GeoJson(gdf_crossings, style_function=lambda x: {"color": "red", "weight": 4}).add_to(map)

# Plot map
template = plot_utils.gen_template_curbs()
macro = MacroElement()
macro._template = Template(template)
map.get_root().add_child(macro)
map